

This notebook contains an example for teaching.


# A Simple Case Study using Wage Data from 2015 - proceeding

So far we considered many machine learning method, e.g Lasso and Random Forests, to build a predictive model. In this lab, we extend our toolbox by predicting wages by a neural network.

## Data preparation

Again, we consider data from the U.S. March Supplement of the Current Population Survey (CPS) in 2015.



This notebook contains an example for teaching.


# A Simple Case Study using Wage Data from 2015 - proceeding

So far we considered many machine learning method, e.g Lasso and Random Forests, to build a predictive model. In this lab, we extend our toolbox by predicting wages by a neural network.

## Data preparation

Again, we consider data from the U.S. March Supplement of the Current Population Survey (CPS) in 2015.

In [2]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [3]:
rdata_read = pyreadr.read_r("../data/wage2015_subsample_inference.Rdata")
data = rdata_read[ 'data' ]
n = data.shape[0]

type(data)


pandas.core.frame.DataFrame

In [4]:
# Import relevant packages for splitting data
import random
import math



# Set Seed
# to make the results replicable (generating random numbers)
np.random.seed(0)
random = np.random.randint(0, data.shape[0], size=math.floor(data.shape[0]))
data["random"] = random
random    # the array does not change 

array([2732, 2607, 1653, ..., 4184, 2349, 3462])

In [5]:
data_2 = data.sort_values(by=['random'])
data_2.head()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,...,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2,random
rownames,,,,,,,,,,,,,,,,,,,,,
2223,26.442308,3.274965,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,29.0,8.4100,24.389000,70.728100,340,1,8660,20,0
3467,19.230769,2.956512,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,33.5,11.2225,37.595375,125.944506,9620,22,1870,5,0
13501,48.076923,3.872802,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,2.0,0.0400,0.008000,0.001600,3060,10,8190,18,0
15588,12.019231,2.486508,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,29.0,8.4100,24.389000,70.728100,6440,19,770,4,2
16049,39.903846,3.686473,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,12.0,1.4400,1.728000,2.073600,1820,5,7860,17,2


In [6]:
# Create training and testing sample 
data_train = data_2[ : math.floor(n*3/4)]    # training sample
data_test =  data_2[ math.floor(n*3/4) : ]   # testing sample
print(data_train.shape)
print(data_test.shape)

(3862, 21)
(1288, 21)


In [7]:
data_train = data_train.iloc[:, 0:16]
data_test = data_test.iloc[:, 0:16] 
data_test

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4
rownames,,,,,,,,,,,,,,,,
9210,29.807692,3.394766,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,4.84,10.648,23.4256
16484,43.269231,3.767442,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,17.0,2.89,4.913,8.3521
16448,24.038462,3.179655,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,11.0,1.21,1.331,1.4641
27392,10.097115,2.312250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,30.0,9.00,27.000,81.0000
11596,8.653846,2.158004,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.36,0.216,0.1296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27533,21.634615,3.074295,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10.0,1.00,1.000,1.0000
7218,13.461538,2.599837,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,28.0,7.84,21.952,61.4656
7204,27.403846,3.310683,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.16,0.064,0.0256


In [8]:
# normalize the data

scaler = preprocessing.StandardScaler().fit(data_train)
scaler = preprocessing.StandardScaler().fit(data_test)

data_train_scaled = scaler.transform(data_train)
data_test_scaled = scaler.transform(data_test)


In [9]:
columns = list(data_train)

In [10]:
data_train_scaled = pd.DataFrame(data_train_scaled, columns = columns)
data_test_scaled = pd.DataFrame(data_test_scaled, columns = columns)
data_test_scaled

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4
0,0.333677,0.755193,1.134788,-0.15177,1.806904,-0.620440,-0.706695,-0.389945,1.633865,-0.640885,-0.527046,-0.529451,0.749646,0.431021,0.147658,-0.046012
1,1.018171,1.425444,1.134788,-0.15177,-0.553433,-0.620440,1.415037,-0.389945,-0.612046,1.560342,-0.527046,-0.529451,0.279612,-0.051673,-0.243088,-0.323785
2,0.040322,0.368319,1.134788,-0.15177,-0.553433,-0.620440,1.415037,-0.389945,-0.612046,1.560342,-0.527046,-0.529451,-0.284429,-0.467532,-0.487143,-0.450716
3,-0.668570,-1.191697,-0.881222,-0.15177,1.806904,-0.620440,-0.706695,-0.389945,-0.612046,-0.640885,1.897367,-0.529451,1.501701,1.460768,1.261777,1.014963
4,-0.741957,-1.469106,1.134788,-0.15177,1.806904,-0.620440,-0.706695,-0.389945,1.633865,-0.640885,-0.527046,-0.529451,-0.754463,-0.677937,-0.563112,-0.475308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,-0.081909,0.178829,1.134788,-0.15177,-0.553433,-0.620440,1.415037,-0.389945,-0.612046,-0.640885,1.897367,-0.529451,-0.378436,-0.519515,-0.509695,-0.459268
1284,-0.497495,-0.674476,-0.881222,-0.15177,1.806904,-0.620440,-0.706695,-0.389945,1.633865,-0.640885,-0.527046,-0.529451,1.313687,1.173627,0.917839,0.654985
1285,0.211446,0.603971,1.134788,-0.15177,-0.553433,-0.620440,-0.706695,2.564463,1.633865,-0.640885,-0.527046,-0.529451,-0.942477,-0.727445,-0.573468,-0.477224
1286,-0.333039,-0.287225,1.134788,-0.15177,-0.553433,1.611758,-0.706695,-0.389945,-0.612046,-0.640885,-0.527046,1.888750,1.313687,1.173627,0.917839,0.654985


Then, we construct the inputs for our network.

In [11]:
formula_basic = "lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we"
Y_train, model_X_basic_train = patsy.dmatrices(formula_basic, data_train_scaled, return_type='dataframe')
Y_test, model_X_basic_test = patsy.dmatrices(formula_basic, data_test_scaled, return_type='dataframe')

### Neural Networks

First, we need to determine the structure of our network. We are using the R package *keras* to build a simple sequential neural network with three dense layers.

In [12]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim = model_X_basic_train.shape[1], activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1))

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [14]:
# compile the keras model
opt = keras.optimizers.Adam(learning_rate=0.005)
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None)

Let us have a look at the structure of our network in detail.

In [15]:
model.compile(loss=mse, optimizer= opt , metrics=mae)
model.summary(line_length=None, positions=None, print_fn=None)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                220       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________


It is worth to notice that we have in total $441$ trainable parameters.

Now, let us train the network. Note that this takes some computation time. Thus, we are using gpu to speed up. The exact speed-up varies based on a number of factors including model architecture, batch-size, input pipeline complexity, etc.

In [16]:
# fit the keras model on the dataset
num_epochs = 1000

In [17]:
# fit the keras model on the dataset
model.fit(model_X_basic_train, Y_train, epochs=150, batch_size=10)

Epoch 1/150
387/387 [==============================] - 0s 781us/step - loss: 0.9216 - mean_absolute_error: 0.7322
Epoch 2/150
387/387 [==============================] - 0s 711us/step - loss: 0.8893 - mean_absolute_error: 0.7221
Epoch 3/150
387/387 [==============================] - 0s 475us/step - loss: 0.8742 - mean_absolute_error: 0.7139
Epoch 4/150
387/387 [==============================] - 0s 480us/step - loss: 0.8752 - mean_absolute_error: 0.7134
Epoch 5/150
387/387 [==============================] - 0s 473us/step - loss: 0.8657 - mean_absolute_error: 0.7067
Epoch 6/150
387/387 [==============================] - 0s 472us/step - loss: 0.8677 - mean_absolute_error: 0.7106
Epoch 7/150
387/387 [==============================] - 0s 507us/step - loss: 0.8627 - mean_absolute_error: 0.7082
Epoch 8/150
387/387 [==============================] - 0s 454us/step - loss: 0.8623 - mean_absolute_error: 0.7078
Epoch 9/150
387/387 [==============================] - 0s 433us/step - loss: 0.8554 - me

Epoch 70/150
387/387 [==============================] - 0s 472us/step - loss: 0.8259 - mean_absolute_error: 0.6925
Epoch 71/150
387/387 [==============================] - 0s 430us/step - loss: 0.8276 - mean_absolute_error: 0.6923
Epoch 72/150
387/387 [==============================] - 0s 461us/step - loss: 0.8256 - mean_absolute_error: 0.6939
Epoch 73/150
387/387 [==============================] - 0s 423us/step - loss: 0.8270 - mean_absolute_error: 0.6928
Epoch 74/150
387/387 [==============================] - 0s 425us/step - loss: 0.8258 - mean_absolute_error: 0.6923
Epoch 75/150
387/387 [==============================] - 0s 487us/step - loss: 0.8297 - mean_absolute_error: 0.6941
Epoch 76/150
387/387 [==============================] - 0s 423us/step - loss: 0.8250 - mean_absolute_error: 0.6912
Epoch 77/150
387/387 [==============================] - 0s 425us/step - loss: 0.8258 - mean_absolute_error: 0.6933
Epoch 78/150
387/387 [==============================] - 0s 433us/step - loss: 0.

387/387 [==============================] - 0s 441us/step - loss: 0.8158 - mean_absolute_error: 0.6890
Epoch 141/150
387/387 [==============================] - 0s 423us/step - loss: 0.8189 - mean_absolute_error: 0.6909
Epoch 142/150
387/387 [==============================] - 0s 428us/step - loss: 0.8201 - mean_absolute_error: 0.6909
Epoch 143/150
387/387 [==============================] - 0s 427us/step - loss: 0.8170 - mean_absolute_error: 0.6900
Epoch 144/150
387/387 [==============================] - 0s 424us/step - loss: 0.8184 - mean_absolute_error: 0.6907
Epoch 145/150
387/387 [==============================] - 0s 425us/step - loss: 0.8161 - mean_absolute_error: 0.6901
Epoch 146/150
387/387 [==============================] - 0s 439us/step - loss: 0.8170 - mean_absolute_error: 0.6909
Epoch 147/150
387/387 [==============================] - 0s 459us/step - loss: 0.8180 - mean_absolute_error: 0.6921
Epoch 148/150
387/387 [==============================] - 0s 430us/step - loss: 0.8143 

In [20]:
model.metrics_names
model.evaluate(model_X_basic_test, Y_test, verbose = 0)

[0.8773821592330933, 0.7171093821525574]

In [19]:
pred_nn = model.predict(model_X_basic_test)
pred_nn

array([[-0.46916604],
       [ 0.37125787],
       [ 0.05024599],
       ...,
       [ 0.20264128],
       [ 0.3821144 ],
       [ 0.11370236]], dtype=float32)

In [130]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [133]:
resid_basic = (Y_test-pred_nn)**2

MSE_nn_basic = sm.OLS( resid_basic , np.ones( resid_basic.shape[0] ) ).fit().summary2().tables[1].iloc[0, 0:2]
MSE_nn_basic

Coef.       0.846993
Std.Err.    0.046878
Name: const, dtype: float64

In [141]:
R2_nn_basic = 1 - ( MSE_nn_basic[0]/Y_test.var() )
print( f"The R^2 using NN is equal to = {R2_nn_basic[0]}" ) # MSE NN (basic model) 

The R^2 using NN is equal to, 0.15366423475210123
